# Load books

In [13]:
import glob
import os
books_directory = '/home/ubuntu/Downloads/twiker-experiments/process_analysis/warpeace/books'
books = []

# List all .txt files in the directory using glob
book_files = glob.glob(os.path.join(books_directory, '*.txt'))
# Iterate over each file in the directory
for book_file in book_files:
    with open(book_file, 'r',encoding='UTF-8') as fs:
        books.append(fs.read())
    print(f'{os.path.basename(book_file)}', len(books[-1]))
print()

book-war-and-peace.txt 3202303



In [14]:
import re
def clean_text(text):
    """Clean the text by removing special characters, tags, etc."""
    # Replace newlines and carriage returns with a space
    text = text.replace('\n\n', '\n')
    text = re.sub('<[^>]+>', ' ', text)
    return text

cleaned_books =[clean_text(book) for book in books]

In [15]:
# Some long paragraphs will be truncated during training. 
# To avoid lossing data, we split them into shorter ones.

import nltk
from transformers import GPT2Tokenizer

# Download the necessary NLTK data
nltk.download('punkt')

# Initialize the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def split_text_at_sentences(text, max_tokens=500):
    # Tokenize the text into sentences
    sentences = nltk.tokenize.sent_tokenize(text)
    
    chunks = []
    current_chunk = []
    current_tokens = []
    
    for sentence in sentences:
        # Tokenize the sentence
        sentence_tokens = tokenizer.encode(sentence, add_special_tokens=False)
        
        # Check if adding this sentence would exceed the max token limit
        if len(current_tokens) + len(sentence_tokens) > max_tokens:
            # If so, finalize the current chunk and start a new one
            chunks.append(' '.join(current_chunk))
            current_chunk = []
            current_tokens = []
        
        # Add the sentence to the current chunk
        current_chunk.append(sentence)
        current_tokens.extend(sentence_tokens)
    
    # Add the last chunk if there's any content left
    if current_chunk:
        chunks.append(' '.join(current_chunk))
    
    return chunks

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
chucked_books = []
for i, book in enumerate(cleaned_books):
    chunked_book = ''
    chunks = split_text_at_sentences(book.replace("\n", " "))
    for chunk in chunks:
        chunked_book += chunk + "\n"
    chucked_books.append(chunked_book)

# Merge to train and eval

In [17]:
# Assuming `chucked_books` is a list where each element is a string containing paragraphs of the book.
book_content = chucked_books[0]  # Get the entire book content
 
# Assuming paragraphs are already defined (e.g., separated by '\n\n').
paragraphs = book_content.split('\n')
 
# Calculate the split index for 80% of the paragraphs for training
split_index = int(0.65 * len(paragraphs))
 
# Check if split_index is at a paragraph boundary
train_paragraphs = paragraphs[:split_index]  # First 80% of the paragraphs for training
eval_paragraphs = paragraphs[split_index:]   # Remaining 20% of the paragraphs for evaluation
 
# Ensure that paragraphs are not split inappropriately
# In this case, since we split by paragraph, each set will contain whole paragraphs.
 
# Join the paragraphs back into a single string for train and eval
train = '\n'.join(train_paragraphs)
eval = '\n'.join(eval_paragraphs)
 
# Write the train, eval, and full content to respective files
with open('data_files/train.txt', 'w') as fs:
    fs.write(train)
 
with open('data_files/eval.txt', 'w') as fs:
    fs.write(eval)
 
with open('data_files/full.txt', 'w') as fs:
    fs.write(book_content)

In [12]:
def count_lines_in_file(filename):
    with open(filename, 'r') as file:
        line_count = sum(1 for line in file)
    return line_count
 
# Example usage:
filename = 'your_text_file.txt'
train_number_of_lines = count_lines_in_file('data_files/train.txt')
full_number_of_lines = count_lines_in_file('data_files/full.txt')
eval_number_of_lines = count_lines_in_file('data_files/eval.txt')
print(f'The train file has {train_number_of_lines} lines.')
print(f'The full file has {full_number_of_lines} lines.')
print(f'The eval file has {eval_number_of_lines} lines.')

The train file has 968 lines.
The full file has 1489 lines.
The eval file has 521 lines.
